In [2]:
# Benders decomposition for Fixed Charge Transportation Problem

import gurobipy as gp
from gurobipy import *
import numpy as np


In [9]:
capacity = [5,8,7]
demand = [3,6,6,5]

locations = len(capacity)
customers = len(demand)

varcost = [[4,5,2,7],[5,8,6,2],[8,9,4,3]]
fixcost = [[2,10,1,5],[7,2,7,8],[4,0,3,9]]

In [10]:
bigM = [[min(capacity[i], demand[j]) for j in range(customers)] for i in range(locations)]
bigM

[[3, 5, 5, 5], [3, 6, 6, 5], [3, 6, 6, 5]]

In [11]:

file = open('43.txt','r')
data = file.readlines()

[locations, customers] = [int(x) for x in data[0].split(' ')]
capacity = [int(data[1].split(' ')[i]) for i in range(locations) ]
demand = [int(data[2].split(' ')[j]) for j in range(customers) ]

fixcost = [ [ int(data[i+3].split('   ')[j]) for j in range(customers) ] for i in range(locations)]
varcost = [ [ int(data[i+3+locations+1].split('    ')[j]) for j in range(customers) ] for i in range(locations)]

In [12]:
bigM = [[min(capacity[i], demand[j]) for j in range(customers)] for i in range(locations)]
bigM

[[20, 19, 19, 18, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [20, 19, 19, 18, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [20, 19, 19, 18, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [18, 18, 18, 18, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [18, 18, 18, 18, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [17, 17, 17, 17, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [17, 17, 17, 17, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 7, 5, 5, 5, 1],
 [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 7, 5, 5, 5, 1],
 [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 7, 5, 5, 5, 1],
 [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 7, 5, 5, 5, 1],
 [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1],
 [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1]]

In [13]:
# Define Benders subproblem
        
def subproblem(model, where):
    if where == GRB.Callback.MIPSOL:
        
        v_y = model.cbGetSolution(model._y)
        #print('\nCurrent y: ', v_y)
        
        LB = model.cbGetSolution(model._z)
        print('Current LB: ', LB, '\n')

        print(model.cbGet(GRB.Callback.MIPSOL_OBJBND))
        
        bsp = gp.Model("Subproblem") 
        #variables
        x = {}
        for i in range(locations):
            for j in range(customers):
                x[i,j] = bsp.addVar(obj=varcost[i][j])
        
        #demand constraint
        demand_constr = {} 
        for j in range(customers):
            demand_constr[j] = bsp.addConstr(sum(x[i,j] for i in range(locations)) >= demand[j])
        
        #capacity constraint
        cap_constr = {}
        for i in range(locations):
            cap_constr[i] = bsp.addConstr(sum(x[i,j] for j in range(customers)) <= capacity[i])
        
        #logical constraint
        log_constr = {}
        for i in range(locations):
            for j in range(customers):
                log_constr[i,j] = bsp.addConstr(x[i,j] <= v_y[i,j] * bigM[i][j])
        
        bsp.optimize()
        
        
        print('\n\n',bsp.Status,'\n\n')
        
        if bsp.Status == 2:
         #feasible subproblem
            #update upper bound
            fc = sum(fixcost[i][j]*v_y[i,j] for i in range(locations) for j in range(customers))
            model.addConstr(model._z <= fc + bsp.objVal)
            
            v = np.zeros(customers) #dual of demand constraints
            for j in range(customers):
                v[j] = demand_constr[j].pi #get dual value

            w = np.zeros((locations,customers)) #dual of logical constraints
            for i in range(locations):
                for j in range(customers):
                    w[i,j] = log_constr[i,j].pi

            u = np.zeros(locations) #dual of capacity constraints
            for i in range(locations):
                u[i] = cap_constr[i].pi

            cm = np.zeros((locations,customers)) #coefficient of y in master problem
            for i in range(locations):
                for j in range(customers):
                    cm[i,j] = fixcost[i][j] + bigM[i][j] * w[i,j]

                
            model.cbLazy( model._z >= sum(demand[j] * v[j] for j in range(customers)) + 
                         sum(capacity[i] * u[i] for i in range(locations)) + 
                         sum(cm[i,j] * model._y[i,j] for i in range(locations) for j in range(customers))) 
            

                
                # save result of x from subproblem
            global transport
            for i in range(locations):
                for j in range(customers):
                    transport[i,j] = x[i,j].x
            print(transport)


        else:
            infbsp = gp.Model('Infeasible subproblem')
            dummy = infbsp.addVar(obj = 1)
            
            v = {}
            for j in range(customers): v[j] = infbsp.addVar(lb=0)
            w = {}
            for i in range(locations):
                for j in range(customers):
                    w[i,j] = infbsp.addVar(lb=0)
            u = {}
            for i in range(locations): u[i] = infbsp.addVar(lb=0)
            
            infbsp.addConstr( sum(demand[j]*v[j] for j in range(customers)) -  
                              sum(capacity[i]*u[i] for i in range(locations)) -  
                              sum(bigM[i][j]*v_y[i,j]*w[i,j] for i in range(locations) for j in range(customers)) == 1 )                
            for i in range(locations):
                for j in range(customers):
                    infbsp.addConstr( v[j] - u[i] - w[i,j] <= 0);
            infbsp.addConstr(dummy == 0)
            
            #print(infbsp.Status)

            infbsp.modelSense = -1 #set to maximization
            infbsp.optimize()
           
            #positive dual variables => use - instead of +
            model.cbLazy( sum(demand[j] * v[j].x for j in range(customers)) -
                sum(capacity[i] * u[i].x for i in range(locations)) -  
                sum(bigM[i][j]*w[i,j].x*model._y[i,j] for i in range(locations) for j in range(customers)) <= 0)
                   
                   
        bsp.dispose()
                       

In [ ]:

# Masterproblem
                     
m = gp.Model("Benders Fixed Charge Transportation Masterproblem")

y = {}
z = m.addVar(obj=1)                                                                           
for i in range(locations):
    for j in range(customers):
        y[i,j] = m.addVar(vtype=GRB.BINARY)

for j in range(customers):
    m.addConstr(sum(y[i,j] for i in range(locations)) >= 1)
        
m.Params.lazyConstraints = 1
m._y = y
m._z = z

transport = np.zeros((locations, customers)) # to save result of x from subproblem
m.optimize(subproblem)
print(m.objBound)

print("Objective: " + str(m.objVal))
for i in range(locations):
    print([y[i,j].x for j in range(customers)])

print("Solution:")
import pandas as pd
index = ['Source ' + str(x) for x in range(locations)]
columns = ['Customer ' + str(x) for x in range(customers)]

solution = pd.DataFrame(index=index, columns=columns)

for i in range(locations):
    for j in range(customers):
        solution.iloc[i,j] = transport[i,j]
        
print(solution)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 15 rows, 226 columns and 225 nonzeros
Model fingerprint: 0x36fa8909
Variable types: 1 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Current LB:  0.0 

-1e+100
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0xdf51283b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 225 rows and 0 columns
Presolve time: 0.00s
Presolved: 30 rows, 225 columns, 450 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.150000e+01   0.000

Infeasible model


 3 


Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 227 rows, 256 columns and 723 nonzeros
Model fingerprint: 0x59c4bc54
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 227 rows and 256 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  0.0 

0.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0x735bfe9e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 225 rows and 207 

Current LB:  0.0 

0.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0x12e1cbde
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 242 rows and 206 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model


 3 


Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 227 rows, 256 columns and 732 nonzeros
Model fingerprint: 0xe5bdf13b
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 207 rows and 222 columns
Presolve time: 0.01s
Presolved: 20 rows, 34 columns, 90 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   1.000000e-01   0.000000e+00      0s


  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 192 rows and 203 columns
Presolve time: 0.00s
Presolved: 35 rows, 53 columns, 153 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   1.000000e-01   0.000000e+00      0s
      29   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 29 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  0.0 

0.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0xefca0ba7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 237 rows and 191 columns
Presolve time: 0.01s
Presolved: 18 rows, 34 columns, 67 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.57700

Model fingerprint: 0x244e89f9
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 227 rows and 256 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  9776.0 

8780.940789473683
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0xa1f239a2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 225 rows and 201 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds
Infeasible model


 3 


Gurobi Optimizer version 9.0.

...

  Matrix range     [1e-13, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 205 rows and 218 columns
Presolve time: 0.01s
Presolved: 22 rows, 38 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
      18   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 18 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  13059.00000000001 

12746.239629554853
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0xe5ca1460
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-14, 2e+01]
Presolve removed 240 rows and 212 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model


 3 




  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-17, 2e+01]
Presolve removed 239 rows and 212 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.01 seconds
Infeasible model


 3 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 227 rows, 256 columns and 729 nonzeros
Model fingerprint: 0x2f429008
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 210 rows and 225 columns
Presolve time: 0.00s
Presolved: 17 rows, 31 columns, 75 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   1.052632e-01   0.000000e+00      0s
      14   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  13053.999999999996 

12750.89


Solved in 11 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  13033.0 

12756.028060645842
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0xcacd84d4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-16, 2e+01]
Presolve removed 241 rows and 212 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model


 3 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 227 rows, 256 columns and 729 nonzeros
Model fingerprint: 0x528dc77a
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 210 rows and 225 columns
Presolve time: 0.01s

Model fingerprint: 0x564fe15a
Coefficient statistics:
  Matrix range     [5e-13, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 212 rows and 229 columns
Presolve time: 0.00s
Presolved: 15 rows, 27 columns, 64 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   1.052632e-01   0.000000e+00      0s
      10   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  13040.0 

12767.970227691496
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0x722d44e0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-18, 2e+01]
Presolve removed 245 rows and 211 columns
Presolve time: 0.00s

Solved in 0 iteration

Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-16, 2e+01]
Presolve removed 245 rows and 212 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model


 3 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 227 rows, 256 columns and 730 nonzeros
Model fingerprint: 0xeee89b77
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 210 rows and 226 columns
Presolve time: 0.01s
Presolved: 17 rows, 30 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   5.000000e-02   0.000000e+00      0s
      10   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations an

In [5]:
help(gp.Model.optimize)


    ROUTINE:
      optimize()

    PURPOSE:
      Optimize the model.

    ARGUMENTS:
      None.

    RETURN VALUE:
      None.

    EXAMPLE:
      model.optimize()

    NOTES:
      The algorithm used to optimize the model depends on the model type and
      on the parameter settings.  A MIP model will always be optimized using
      the branch-and-cut algorithm.  A continuous model will be optimized
      using the dual simplex algorithm by default; the Method parameter
      can be used to choose a different algorithm.
    
